In [34]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm import tqdm

In [29]:
base_dir = r"C:\Users\M47h1\Documents\EiffelMedtech\pytorch_cyclegan\datasets\xray2drr\trainA"
output_dir = r"C:\Users\M47h1\Documents\EiffelMedtech\pytorch_cyclegan\datasets\xray2drr\bone_removed"

# iterate over all images in the directory

l_basenames = []
for filename in os.listdir(base_dir):
    if filename.endswith(".png"):

        add = filename.split(".")[0].split("_")[-1]
        if add != "0":
            continue
        print(filename)
        image = cv2.imread(os.path.join(base_dir, filename), cv2.IMREAD_GRAYSCALE)
            
        # Apply thresholding to create a binary image
        _, binary_image = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY)
        
        # morphological closing
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
        

            
        # Find contours
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print(len(contours))
        if len(contours) == 1:
            # Save the processed image
            cv2.imwrite(os.path.join(output_dir, filename ), image)
            continue
        
        # sort contours by area
        contours = sorted(contours, key=cv2.contourArea)
        largest_contour = contours[-1]
        second_largest_contour = contours[-2]
        
        # continue if second largest contour is smaller than 1/4 of the largest contour
        print(cv2.contourArea(second_largest_contour) / cv2.contourArea(largest_contour))
        if cv2.contourArea(second_largest_contour) < cv2.contourArea(largest_contour) / 4:
            cv2.imwrite(os.path.join(output_dir, filename), image)
            continue

        
        # Draw the smallest contour with black color to remove it
        cv2.drawContours(image, [second_largest_contour], -1, (0, 0, 0), thickness=cv2.FILLED)
        
        # Save the processed image
        cv2.imwrite(os.path.join(output_dir, filename), image)

            

ALL-PIE1_cropSeg_LF_135_0.png
2
0.83621557532532
ALL-PIE1_cropSeg_LF_45_0.png
1
ALL-PIE1_cropSeg_RF_135_0.png
1
ALL-PIE1_cropSeg_RF_45_0.png
1
ALL-PIE_cropSeg_LF_135_0.png
1
ALL-PIE_cropSeg_LF_45_0.png
1
ALL-PIE_cropSeg_RF_135_0.png
1
ALL-PIE_cropSeg_RF_45_0.png
1
ALV-MAR_cropSeg_LF_135_0.png
2
0.9105077445810408
ALV-MAR_cropSeg_LF_45_0.png
1
ALV-MAR_cropSeg_RF_135_0.png
1
ALV-MAR_cropSeg_RF_45_0.png
1
AND-ALA_cropSeg_LF_135_0.png
1
AND-ALA_cropSeg_LF_45_0.png
1
AND-ALA_cropSeg_RF_135_0.png
1
AND-ALA_cropSeg_RF_45_0.png
1
AND-MAR1_cropSeg_LF_135_0.png
2
0.907726763411394
AND-MAR1_cropSeg_LF_45_0.png
1
AND-MAR1_cropSeg_RF_135_0.png
1
AND-MAR1_cropSeg_RF_45_0.png
1
AND-MAR_cropSeg_LF_135_0.png
2
0.7488952636627415
AND-MAR_cropSeg_LF_45_0.png
1
AND-MAR_cropSeg_RF_135_0.png
1
AND-MAR_cropSeg_RF_45_0.png
1
AUJ-LOU_cropSeg_LF_135_0.png
2
0.7828419204172314
AUJ-LOU_cropSeg_LF_45_0.png
1
AUJ-LOU_cropSeg_RF_135_0.png
1
AUJ-LOU_cropSeg_RF_45_0.png
2
0.5034159932992327
AujLo_cropSeg_LF_45_0.png
1

In [35]:


def get_contours_bone(img_orig, img_output):
    contours, _ = cv2.findContours(img_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    black_img = np.zeros_like(img_output)
    contour_max = max(contours, key=cv2.contourArea)
    contour_img = cv2.drawContours(black_img.copy(), [contour_max], -1, 255, 1)

    contour_on_orig = cv2.drawContours(img_orig.copy(), [contour_max], -1, (255, 0, 0), 1)
    contour_on_orig = cv2.cvtColor(contour_on_orig.copy(), cv2.COLOR_GRAY2RGB)

    return contour_img, contour_on_orig


def get_smoother_masked_image(mask_img, img, contour_img):
    contour_img_blur = cv2.GaussianBlur(contour_img, (3, 3), 7)
    max_contour_blur = 130  # avoid too high values where contours are close to each other
    contour_img_blur[contour_img_blur > max_contour_blur] = max_contour_blur

    scale_factor = 254.0 / max_contour_blur
    contour_img_blur = (contour_img_blur * scale_factor).astype(np.uint8)
    # bitwise or the mask and the contour image

    mask_img = cv2.bitwise_or(mask_img, contour_img_blur)
    new_mask_norm = mask_img / 255
    unique, counts = np.unique(new_mask_norm, return_counts=True)
    smooth_img = img * new_mask_norm
    return smooth_img


def get_true_ratio(img, true_size, resized_size):
    true_height, true_width = true_size
    resized_height, resized_width = resized_size
    true_ratio = true_height / true_width
    new_height = resized_height
    new_width = round(new_height / true_ratio)
    img_true_ratio = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    return img_true_ratio


def output_to_squared_image(img, img_size):
    y_nonzero = np.where(img > 0)[0]
    y_min = y_nonzero.min()
    y_max = y_nonzero.max()
    new_shape = y_max - y_min
    cropped_image = img[y_min:y_max, :]
    missing_pixels = new_shape - cropped_image.shape[1]
    assert missing_pixels >= 0, 'width greater than height, should not happen'
    padding = np.zeros((new_shape, missing_pixels // 2))
    # add padding to the left and right of the image without using hstack
    cropped_image = np.concatenate((padding, cropped_image, padding), axis=1)

    # resize the image to the desired size
    cropped_image = cv2.resize(cropped_image, (img_size, img_size), interpolation=cv2.INTER_AREA)

    return cropped_image


In [36]:
output_dir = r"C:\Users\M47h1\Documents\EiffelMedtech\pytorch_cyclegan\datasets\xray2drr\bone_removed"
square_dir = r"C:\Users\M47h1\Documents\EiffelMedtech\pytorch_cyclegan\datasets\xray2drr\squared"

# iterate on output images
for filename in tqdm(os.listdir(output_dir)):
    if filename.endswith(".png"):
        # read image
        img_output = cv2.imread(os.path.join(output_dir, filename), cv2.IMREAD_GRAYSCALE)

        squared_output = output_to_squared_image(img_output, 256)
        cv2.imwrite(os.path.join(square_dir, filename), squared_output)
        